In [ ]:
#Preprocessing video -> image frames
import cv2
import os
import numpy as np
import imageio

#Open the video file
capture = cv2.VideoCapture('./datasets/คนละทีเดียวกัน.mp4')
i = 0
frameNumber = 1
while(capture.isOpened()):
    ret, frame = capture.read()
    if ret == False:
        break
    if i % 3 == 0:
        cv2.imwrite('./datasets/คนละทีเดียวกัน/คนละทีเดียวกัน'+str(frameNumber)+'.jpg', frame)
        print('./datasets/คนละทีเดียวกัน/คนละทีเดียวกัน'+str(frameNumber)+'.jpg')
        frameNumber = frameNumber+1
    i = i+1
capture.release()
cv2.destroyAllWindows()

In [ ]:
#Resize image
import os
from PIL import Image
import numpy as np
import imageio
count = 1
for path in sorted(os.listdir('./datasets/เพราะเรา(ยัง)คู่กัน')):
    img = Image.open('./datasets/เพราะเรา(ยัง)คู่กัน/' + path)
    img = img.resize((224,224), Image.NEAREST)
    img = np.array(img)
    imageio.imwrite('./datasets/2/' + str(count) + '.jpg', img)
    print('./datasets/2/' + str(count) + '.jpg')
    count = count + 1

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications.resnet50 import ResNet50

x_train = []
y_train = []
for path in os.listdir('./datasets/1/'):
    img = Image.open('./datasets/1/'+path)
    x_train.append(np.array(img))
    y_train.append(1)
    img.close()


In [ ]:
from tensorflow.keras.optimizers import SGD
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [ ]:
step_size = 19647 // 64
with strategy.scope():
    base_model = ResNet50(include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    prediction = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=prediction)
    optimize = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(loss='binary_crossentropy', optimizer=optimize, metrics=['accuracy'])

In [ ]:
x_train = np.array(x_train)
x_train = x_train.astype(np.float32)
x_train = x_train / 255.0
y_train = np.array(y_train)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(1000).batch(BATCH_SIZE)

In [ ]:
model.fit(train_data, epochs=10)
model.save('weights.h5')

In [ ]:
import imutils
import time
import numpy as np
from PIL import Image
import cv2
import os
import imageio
import copy
from keras.models import load_model
capture = cv2.VideoCapture('datasets/คนละทีเดียวกัน.mp4')
success = 1

model = load_model('weights.h5')
while (True):
  ret, frame = capture.read()
  videoFrame = Image.fromarray(frame)
  videoFrame = videoFrame.resize((800,400), Image.NEAREST)
  videoFrame = np.array(videoFrame)
  videoFrame = videoFrame.astype(np.float32)
  videoFrame /= 255.0
  
  if (type(frame) != 'None'):
    frame = Image.fromarray(frame)
    frame = frame.resize((220,220), Image.NEAREST)
    frame = np.array(frame)
    frame = frame.astype(np.float32)
    frame /= 255.0
    pred_frame = np.expand_dims(frame, axis=0)
    label = model.predict(pred_frame)
    predict = ""
    print(label[0], end="")
    if (label[0][0] > label[0][1]):
        predict = "I'm tee. Me too"
        print(" -> คนละทีเดียวกัน")
    else:
        predict = "Still 2gether"
        print(" -> เพราะเรา(ยัง)คู่กัน")
    font = cv2.FONT_HERSHEY_SIMPLEX 

    # inserting text on video 
    cv2.putText(videoFrame,  predict,  (50, 50),  font, 1,  (0, 255, 0), 2, cv2.LINE_4) 
    
  
    # Display the resulting frame 
    cv2.imshow('Series recognition', videoFrame) 
    

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        capture.release() 
        cv2.destroyAllWindows() 
        break
# release the cap object 
capture.release() 
# close all windows 
cv2.destroyAllWindows() 

